
<h3><font color='black'>Data Mining Project </font></h3>
Gabriel Anand Sundalkar
1001774881

#                          <center style='color:blue'><u> BoardGameGeek Reviews</u></center>  

### Table of Content: 
* [Introduction](#Introduction)
* [Data Exploring](#Data_Exploring)
* [Data Processing](#Data_Processing)
* [Data Exploration](#Data_Exploration)
* [Train Test Split](#train_test_split)

* [Text preprocessing](#Text_preprocessing)
    * [Count Vectorizer](#CountVectorizer) 
    * [TfidfVectorizer (Term Frequency times Inverse Document Frequency)](#TfidfVectorizer)
        
* [Model Selectoin](#Model_Selectoin)
    * [Linear and Logistic Regression](#Linear_and_Logistic_Regression)
    * [Linear Classifiers](#Linear_Classifiers)
    * [Support Vector Machine](#Support_Vector_Machine)
    * [Linear Support Vector Machine](#Linear_Support_Vector_Machine)
    * [Naive Bayes Classifier](#Naive_Bayes_Classifier)
    * [Decision Tree Classifier](#Decision_Tree_Classifier)
    * [Ensemble Model](#Ensemble_Model)
        * [Boosting Models](#Boosting_Models)
        * [Random Forest Classifier](#Random_Forest_Classifier)
        
* [hyperparameter Tuning ](#hyperparameter_Tuning)
     * [Grid Search Cross Validation](#GridSearchCV)
     * [Randomized Search Cross Validation](#RandomizedSearchCV)
        
* [Results](#Results)
* [References](#References)

<h1 id="Introduction" style='color:blue'>Introduction</h1>




The goal of this project is to showcase our dataminning skills by building a classifier

The present work is the result of putting together many posts and code documentation


In [ ]:
import sys
import re 
import sklearn
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn import linear_model
from sklearn import svm
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn import decomposition, ensemble
from sklearn import tree

from sklearn.ensemble import RandomForestClassifier 
from sklearn.model_selection import learning_curve
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import explained_variance_score
from sklearn.model_selection import RandomizedSearchCV

from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error 
from sklearn import linear_model, preprocessing
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn import model_selection, preprocessing, metrics
from sklearn.model_selection import cross_val_score , train_test_split

import nltk
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from collections import defaultdict


<a id = "Data_Description"></a>

<h1 id="Data_Exploring" style='color:blue'>Data Description</h1>



 The source of the data is Kaggle https://www.kaggle.com/jvanelteren/boardgamegeek-reviews.
 
 The data was extracted the 2020-08-19 (As it is constanly updated)
 
 The data spread over 3 CSV files:
 
- <i>games_detailed_info.csv</i>  (with 56 columns and 17063) : 
This file has a many detials about all type of board games, exmaples: are  the game rank, number of plyers,    number of user rated, time requried for the game, desiner and some instructions.    
*** 
-  <i>bgg-13m-reviews.csv</i> (with 6 columns and 13170073):
provids the user names with there reviews and rating for each game.
*** 
- <i>2019-08-19.csv</i> (9 columns and 17065) : 
This file contain more precisis information for each game the game name ,year, rank and number of users rated. 

### Load Dataset

In [ ]:
detailed_df = pd.read_csv('./data/games_detailed_info.csv',low_memory=False)  # for the interpretuer to deterimne datatype it take hight memorey
reviews_df = pd.read_csv('./data/bgg-15m-reviews.csv')
ranking_df = pd.read_csv('./data/2020-08-19.csv')

In [ ]:
 detailed_df.head()

In [ ]:
reviews_df.head()

In [ ]:
ranking_df.head()

In [ ]:
detailed_df.info()

In [ ]:
ranking_df.info()

In [ ]:
reviews_df.info()


<a id = "Data_Processing"></a>

<h1 id="Data_Processing" style='color:blue'> Data Processing </h1>


Data cleaing and data process is the first phase of a data science project, in fact real data is messy, and include missing parts, hence we need to remove and correct the missing data, so that our eventual machine learning model doesn't learn from noise, thus reducing the performance of the end model

we are going to drop all the missing columns and the missing reviews. Then we are going to merge all the data frames together to produce one data frame with all the required features.



In [ ]:
# count the missing data
detailed_df.isnull().sum(), ranking_df.isnull().sum(), reviews_df.isnull().sum()

In [ ]:
# Drop all the coulmns with missing values and the redundent/uneeded once for our classfiaction problem
missing_columns_names = [
    'Unnamed: 0',   
    "Children's Game Rank", 'Commodore 64 Rank','Customizable Rank', 'Family Game Rank','Party Game Rank', 
    'Abstract Game Rank','Accessory Rank', 'Amiga Rank','Arcade Rank', 'Atari ST Rank',                     
    'RPG Item Rank', 'Strategy Game Rank', 'Thematic Rank', 'Video Game Rank',        
    'boardgamedesigner', 'boardgameexpansion',  'boardgamefamily',  'boardgameimplementation', 
    'War Game Rank', 'alternate', 'boardgameartist', 'boardgamecategory', 'boardgamecompilation',             
    'boardgameintegration', 'boardgamemechanic','boardgamepublisher', 'description','image',                               
    'suggested_language_dependence', 'suggested_playerage', 'thumbnail','type' 
]
detailed_df = detailed_df.drop(axis=1, index=None, columns=missing_columns_names)
ranking_df = ranking_df.drop(axis=1, index=None, columns=['URL','Thumbnail'])
reviews_df = reviews_df.drop(axis=1 ,columns=['Unnamed: 0']) 
reviews_df = reviews_df.dropna(axis=0 ,subset=['comment'])    # Drop all missing reviews 



Join the dataframes using common column 'id'



In [ ]:
ranking_detailed_df = pd.merge(left=ranking_df, right=detailed_df, left_on='ID', right_on='id') # inner join

In [ ]:
joint_df = pd.merge(left=reviews_df, right=ranking_detailed_df, how='left', left_on='ID', right_on='ID') # left join
print(f'The joined dataset shape {joint_df.shape}')

### Bivariable analysis

In [ ]:
plt.figure(figsize=(15,8))
sns.heatmap(joint_df.corr())

In [ ]:
plt.figure(figsize=(8, 6))
sns.set(color_codes="True")
sns.distplot(joint_df["Average"])

In [ ]:
plt.figure(figsize=(8, 6))
sns.set(color_codes="True")
sns.distplot(joint_df["rating"])

In [ ]:
plt.figure(figsize=(8, 6))
sns.set(color_codes="True")
sns.distplot(joint_df["averageweight"])

In [ ]:
plt.figure(figsize=(8, 6))
sns.set(color_codes="True")
sns.distplot(joint_df["Rank"])

In [ ]:
plt.figure(figsize=(8, 6))
sns.set(color_codes="True")
sns.distplot(joint_df["numweights"])

In [ ]:
plt.figure(figsize=(8, 6))
sns.set(color_codes="True")
sns.distplot(joint_df["stddev"])

In [ ]:
plt.figure(figsize=(8, 6))
sns.set(color_codes="True")
sns.distplot(joint_df["Bayes average"])

From the above visualization we can:

    1- eleminate variable with negative correlation with target variable
    2- elimianate variables that are strongly correlated (have same information, hence their presence will give more weight to the same information and thus bias the model)

In [ ]:
joint_df['playes_num'] = joint_df['maxplayers'] - joint_df['minplayers']
joint_df['play_time'] = joint_df['maxplaytime'] - joint_df['minplaytime']
joint_df['Time'] = joint_df[['playingtime','play_time']].max(axis=1)

In [ ]:
joint_reviews_renking_detailed = joint_df[['Name','Year','rating','numweights','usersrated', 'numcomments','user','comment', 'wanting', 'wishing', 'trading', 'owned','Time','playes_num','minage']]

In [ ]:
joint_reviews_renking_detailed = joint_reviews_renking_detailed.dropna(axis=0)
joint_reviews_renking_detailed = joint_reviews_renking_detailed.drop_duplicates(subset=None, keep="first", inplace=False)
joint_reviews_renking_detailed.shape

<h1 id="Data_Exploration" style='color:blue'>Data Exploration</h1>

In [ ]:
plt.figure(figsize=(9, 7))
sns.set(color_codes="True")
sns.distplot(joint_reviews_renking_detailed["rating"])

In [ ]:
joint_reviews_renking_detailed.rating.describe()

In [ ]:
rate_group = joint_reviews_renking_detailed.groupby(['rating'])
name_group = joint_reviews_renking_detailed.groupby(['Name'])
year_group = joint_reviews_renking_detailed.groupby(['Year'])
user_group = joint_reviews_renking_detailed.groupby(['user'])

In [ ]:
print(joint_reviews_renking_detailed['rating'].nunique())   
print(joint_reviews_renking_detailed['Name'].nunique()) 
print(joint_reviews_renking_detailed['Year'].nunique()) 
print(joint_reviews_renking_detailed['user'].nunique()) 

In [ ]:
print(joint_reviews_renking_detailed.rating.mode().value_counts())
print(joint_reviews_renking_detailed.Name.mode().value_counts())
print(joint_reviews_renking_detailed.Year.mode().value_counts())
print(joint_reviews_renking_detailed.user.mode().value_counts())

Exploring the top 30 games

In [ ]:
plt.figure(figsize=(15, 7))
joint_reviews_renking_detailed.loc[joint_reviews_renking_detailed['comment'].notna()]['Name'].value_counts()[:30].plot(kind='bar')
plt.xlabel('Name')
plt.ylabel('Review Count')
plt.title('Top 30 Most Reviewed Board Games')
plt.show()


<a id = "Text_preprocessing"></a>

<h1 id="Text_preprocessing" style='color:blue'>Text preprocessing</h1>


The comment column contain expressive text, but we need to process it into a form that can be handled by the model, this is part of the natural language processing (NLP)

To process the text we:

    - Remove the stop words.
    - Remove the punctuations.
    - Convert text to lower case.
    - Drop words with frequency less than 5.


In [ ]:
stop_words = set(stopwords.words('english')) 
replace_signs = re.compile('[/(){}\[\]\|@,;]')    # Remove characteris from dict 
replace_digit = re.compile('[^0-9a-z #+_]')       # Remove digits from dict
words = set(nltk.corpus.words.words())

In [ ]:
# Helpers functions to process text
def tokenize_remove_punctuations(sentences):               
    wordfreq = defaultdict(lambda : 0)
    for sentence in sentences:
        words = nltk.RegexpTokenizer(r"\w+").tokenize(sentence.lower())                                                 # len(word)>2   
        for word in words: 
            if word in stop_words or word.isdigit() or len(word) < 1 or not word.isalpha():
                continue
            wordfreq[word] += 1
    return wordfreq 


def sort_words_freq(frequencies):                        
    return dict(sorted(frequencies.items(), key=lambda word: word[1], reverse=True))
 
def delete_threshold(wordfreq): 
    return {word: frequency for word, frequency in wordfreq.items() if frequency > 5}
  
def vector_matrix_review(X):
    sentence_vectors = []
    for txt,i  in zip(X,y):
        sentence_tokens = nltk.RegexpTokenizer(r"\w+").tokenize(txt)
        sent_vec = []
        for token in wordfreq_sorted_X:   
            if token in sentence_tokens:
                sent_vec.append(1)
            else:
                sent_vec.append(0)
        sentence_vectors.append([sent_vec,i])
    return sentence_vectors    

In [ ]:
x = tokenize_remove_punctuations(joint_reviews_renking_detailed['comment'])
x = sort_words_freq(x)
x = delete_threshold(x)



Let's print the top 50 words in the dictionary,


## Split out Training Set and Test Set 

In [ ]:
X = joint_reviews_renking_detailed['comment'].sample(6000) #frac = 0.10 # this is less than 10% of the dataset 
y = joint_reviews_renking_detailed['rating'].astype(int).sample(6000) # convert all float to a intger for classification 

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 42)


<a id = "CountVectorizer"></a>

# CountVectorizer

In [ ]:
from nltk import word_tokenize          
from nltk.stem import WordNetLemmatizer 


class LemmaTokenizer:
    def __init__(self):
         self.wnl = WordNetLemmatizer()
    def __call__(self, doc):
         return [self.wnl.lemmatize(t) for t in word_tokenize(doc)]

# credits to [https://scikit-learn.org/stable/modules/feature_extraction.html] 

In [ ]:
# convert our text documents to a matrix of token counts
vect = CountVectorizer(min_df=5,lowercase=True,ngram_range=(1, 2), stop_words=stopwords.words('english') + list(string.punctuation))                     
X_train_0 = vect.fit_transform(X_train)
X_test_0 = vect.transform(X_test)


<a id = "TfidfVectorizer"></a>

# TfidfVectorizer (Term Frequency times Inverse Document Frequency)

In [ ]:
tfidf = TfidfVectorizer(ngram_range=(1,2), min_df=5, max_df=0.75, smooth_idf=True, stop_words=stopwords.words('english') + list(string.punctuation))    
X_train_1 = tfidf.fit_transform(X_train)
X_test_1 = tfidf.transform(X_test)      

<h1 id="Model_Selectoin" style='color:blue'>Model Selectoin</h1>

In [ ]:
classifiers = [
    linear_model.LogisticRegression(solver='lbfgs',multi_class='auto',max_iter=1000),
    svm.LinearSVC(),
    MultinomialNB(),
    SGDClassifier(),
    ensemble.AdaBoostClassifier(),
    DecisionTreeClassifier(),
    ensemble.RandomForestClassifier(n_estimators = 100)
]


for item in classifiers:
    print(item)
    model = item
    model.fit(X_train_0, y_train)
    
    y_train_pred = model.predict(X_train_0)
    print("training Score = ",model.score(X_train_0,y_train))   #training score
    print("Train Mean Squer Error =  ", mean_squared_error(y_train_pred.astype(int), y_train))
    
    y_pred = model.predict(X_test_0)
    print("test Score = ", accuracy_score(y_pred.astype(int), y_test))    #test score
    print("Test Mean Squer Error =  ", mean_squared_error(y_pred.astype(int), y_test),"\n")
       
    
    plt.figure(figsize=(10, 8))
    sns.set(color_codes="True")
    sns.distplot(y_test.values[:15], color='b', label= 'Actual', kde_kws={'bw':1.1})   # its shows only the first 15 data point
    sns.distplot(y_pred[:15], color='orange', label='Predicted', kde_kws={'bw':1.1})
    plt.legend()
    plt.show() 
    print("\n---------------------------------------------------------------------------------\n\n")

In [ ]:
classifiers = [
    linear_model.LogisticRegression(solver='lbfgs',multi_class='auto',max_iter=1000), # to handle the FutureWarning
    svm.LinearSVC(),   
    SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42,max_iter=1000, tol=None),
    MultinomialNB(),
    DecisionTreeClassifier(),
    ensemble.AdaBoostClassifier(),
    ensemble.RandomForestClassifier(n_estimators = 100)
]



for item in classifiers:
    print(item)
    model = item
    model.fit(X_train_1, y_train)
    
    y_train_pred = model.predict(X_train_1)
    print("training Score = ",model.score(X_train_1,y_train))   #training score
    print("Train Mean Squer Error =  ", mean_squared_error(y_train_pred.astype(int), y_train))
    
    y_pred = model.predict(X_test_1)
    print("test Score = ", accuracy_score(y_pred.astype(int), y_test))    #test score
    print("Test Mean Squer Error =  ", mean_squared_error(y_pred.astype(int), y_test), "\n")
    
    
    plt.figure(figsize=(10, 8))
    sns.set(color_codes="True")
    sns.distplot(y_test.values[:15], color='b', label= 'Actual', kde_kws={'bw':1.1})
    sns.distplot(y_pred[:15], color='orange', label='Predicted', kde_kws={'bw':1.1})
    plt.legend()
    plt.show() 
    
    
    
    print("\n---------------------------------------------------------------------------------\n\n")


|    | Logistic Regression | Linear support vector classifier | Decision Tree Classifier | RandomForestClassifier |
|:--:|:---------------------:|:---------------------:       |:---------------------:   |:---------------------: |
| Training Score |   0.735   |         0.807                |        0.948             |        0.948            | 
| Test Score     |   0.212   |         0.200                |        0.192              |        0.338            | 

***

|    | Multinomial Niave Bayes |  Stochastic Gradient Descent | AdaBoostClassifier       | 
|:--:|:---------------------:  |:---------------------:       |:---------------------:   |
| Training Score |   0.509     |         0.793                |        0.265             | 
| Test Score     |   0.228     |         0.186                |        0.268             | 




Let's Have a closer look to the numbers

In [ ]:
print("y_test=" ,y_test.values[:15] ,"y_pred=", y_pred[:15])

Predict the first review as an example 

In [ ]:
print(joint_reviews_renking_detailed['comment'][0],"Actual rating is", joint_reviews_renking_detailed['rating'][0])

In [ ]:

# you can try diffrent text to perdict 

model = ensemble.RandomForestClassifier(n_estimators = 100)
model.fit(X_train_1, y_train)
y_pred = model.predict(X_test_1)    
    
y_pred1 = model.predict(tfidf.transform(["Currently, this sits on my list as my favorite game"]))
print("test Score = ",y_pred1.astype(int))



<a id = "hyperparameter_Tuning"></a>

<h1 id="hyperparameter_Tuning" style='color:blue'>hyperparameter Tuning and Model Optimization</h1>

### Cross Validation 

### Training Visualizations and Model Evaluation


Continue with our best model <i>RandomForestClassifier</i>, we are going to look at the confusion matrix, and show the discrepancies between predicted and actual labels. The confusion matrix is a great way to see which categories model is mixing.


In [ ]:

print(metrics.classification_report(y_test, y_pred))
print("Confusion matrix \n", confusion_matrix( y_test, np.array(y_pred))) 


As we can see there are some labels almost are not mentioned 1,2,3,4,5 and 10. Non of these appaer in the prediaction result. Becuase, the model is not good enough, the data is imbalanced and the sample size we took is small. 


<a id ="GridSearchCV"></a>

## Grid Search Cross Validation

In [ ]:

# Parameter estimation using grid search with cross-validation

X = joint_reviews_renking_detailed['comment'].sample(6000) # this is 10% of the dataset 
y = joint_reviews_renking_detailed['rating'].astype(int).sample(6000) # convert all float to a intger for classification 
#split for 30% test set, 70% train set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

tfidf = TfidfVectorizer(ngram_range=(1,2), min_df=5, max_df=0.75, smooth_idf=True, 
                    stop_words=stopwords.words('english') + list(string.punctuation))    
                    
X_trainT = tfidf.fit_transform(X_train)
X_testT = tfidf.transform(X_test)  


from sklearn.model_selection import GridSearchCV

tuned_parameters = [
{'n_estimators': [10, 25], 'max_features': [5, 10], 
 'max_depth': [10, 50, None], 'bootstrap': [True, False]}
]

    
rf = GridSearchCV(RandomForestClassifier(), tuned_parameters, cv=10, scoring='neg_mean_squared_error')
rf.fit(X_trainT, y_train)

#best_estimator_ returns the best estimator chosen by the search
#print(rf.best_estimator_)
print()
print("best_parameters", rf.best_params_)

means = rf.cv_results_['mean_test_score'][rf.best_index_]
stds = rf.cv_results_['std_test_score'][rf.best_index_]
#params = rf.cv_results_['params']
#print("mean , std )" % (mean, std))
print()

y_true, y_pred = y_test, rf.predict(X_testT)
print(classification_report(y_true, y_pred.astype(int)))



The accuracy is still low, 

### Mean squared error (MSE )

Mean squared error measures the average of the squared error between the actual and predicted value.
The closer value to zero the more efficient estimator we have. The value of 2.95 indicates the distance between the actual rating and the predicted rating which still not good enough. 


In [ ]:

grid_best= rf.best_estimator_.predict(X_testT)
MSE = np.square(np.subtract(y_test,y_pred)).mean()   # calculate Mean squared error
print('The model error is', round(MSE, 2),'%')


### Mean Absolute Percentage Error (MAPE)

MAPE is another performance metric similar to MSE, but this one represents percentages.

In [ ]:

grid_best= rf.best_estimator_.predict(X_testT)
errors = abs(grid_best - y_test)
mape = np.mean(100 * (errors / y_test))
accuracy = 100 - mape    
print('The final accuracy of', round(accuracy, 2),'%')



<a id ="RandomizedSearchCV"></a>

## Randomized Search Cross Validation

In [ ]:
# Parameter estimation using Randomized Search with cross-validation


X = joint_reviews_renking_detailed['comment'].sample(6000) # this is 10% of the dataset 
y = joint_reviews_renking_detailed['rating'].astype(int).sample(6000) #frac = 0.7

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

tfidf = TfidfVectorizer(ngram_range=(1,2), min_df=5, max_df=0.75, smooth_idf=True, 
                    stop_words=stopwords.words('english') + list(string.punctuation))    
                    
X_trainTT = tfidf.fit_transform(X_train)
X_testTT = tfidf.transform(X_test)  



# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 20, stop = 200, num = 5)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(1, 45, num = 3)]
# Minimum number of samples required to split a node
min_samples_split = [5, 10]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split}



rf_random = RandomizedSearchCV(estimator = RandomForestClassifier(), 
                               param_distributions = random_grid, n_iter = 5, cv = 10, 
                               verbose=2, random_state=42, scoring='neg_mean_squared_error')
# Fit the random search model
rf_random.fit(X_trainTT, y_train)
random_best= rf_random.predict(X_trainTT)


# best random model 
print(rf_random.best_estimator_)

# best combination of parameters of random search
print(rf_random.best_params_)


#this is the RMSE
final_mse = mean_squared_error(y_train, random_best)
final_rmse = np.sqrt(final_mse)
print('The best model on training set has a RMSE of', round(final_rmse, 2))

    
# Credits goes to     
#https://scikit-learn.org/stable/auto_examples/model_selection/plot_grid_search_digits.html
#https://www.kaggle.com/emanueleamcappella/random-forest-hyperparameters-tuning  

In [ ]:
#import scipy
#scipy.test() clean memory 

In [ ]:
final_model = rf_random.best_estimator_
# Predicting test set results
final_pred = final_model.predict(X_testTT)
final_mse = mean_squared_error(y_test, final_pred)
final_rmse = np.sqrt(final_mse)
print('The final RMSE on the test set is', round(final_rmse, 2))

In [ ]:
#calculate accuracy
errors = abs(final_pred - y_test)
# Calculate mean absolute percentage error (MAPE)
mape = np.mean(100 * (errors / y_test))
# Calculate and display accuracy
accuracy = 100 - mape    
#print result
print('The final model accuracy on the test set is', round(accuracy, 2),'%')

<h1 id="Results" style='color:blue'>Results and Futuer Work</h1>

TfidfVectorizer is more efficatve than using only CounterVectorizer. The most performant machine learing models are: 
* Linear SVC 
* Decision Tree Classifier
* SGD Classifier
* Random Forest Classifier

<h1 id="Challenges" style='color:blue'>Challenges</h1>


We experienced many challenges during this work, usually preparing the data time the most time, then working to improve the model accuracy and tuning the model parameters. The challenged we faced are:

* Understanding data 
* Computation ressources (a larger grid search would need more computation power)
* Feature extraction 
* Improving score



<h1 id="References" style='color:blue'>References</h1>

* https://www.kaggle.com/ngrq94/boardgamegeek-reviews-data-preparation
 
* https://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html
 
* https://monkeylearn.com/text-classification/

* https://www.kaggle.com/emanueleamcappella/random-forest-hyperparameters-tuning

* https://guneetkohli.github.io/machine-learning/board-game-reviews/#.XqK_ky-z1QI

* https://docs.google.com/viewer?a=v&pid=sites&srcid=YWltcy5hYy50enxhaW1zLXRhbnphbmlhLWFyY2hpdmV8Z3g6YmI1ODkxYjEwOGQ4NGVi
